In [ ]:
import numpy as np
import pandas as pd
#import fiona 
#import math
#import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
#import datetime
#import random
#from scipy.interpolate import interp1d
import geopandas as gpd
from scipy.spatial import cKDTree
from shapely.geometry import Point

In [ ]:
hrus_v10_cent_path = 'c:/Users/markstro/work/GIS/hrusAllConusCent/hrusAllConusCent.shp'
hrus_v11_cent_path = 'c:/Users/markstro/work1.1/GIS/hrusAllConusCent/hrusAllConusCent.shp'
hrus_v10_to_v11_mapping_path = 'c:/Users/markstro/work1.1/GIS/hrus_v10_to_v11_mapping_path.shp'

In [ ]:
# Read the version 1.0 shapefile
nhru_v10 = gpd.read_file(hrus_v10_cent_path)

In [ ]:
nhru_v10.head()

In [ ]:
nhru_v10.rename(columns={'nhm_id':'nhm_id_v10'}, inplace=True)

In [ ]:
nhru_v10.head()

In [ ]:
# Read the version 1.1 HRUs
nhru_v11 = gpd.read_file(hrus_v11_cent_path)

In [ ]:
# This gem comes from https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe

def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

In [ ]:
c = ckdnearest(nhru_v11, nhru_v10)

In [ ]:
c["dist_ln"] = np.log(c["dist"])

In [ ]:
c.head()

In [ ]:
c.tail()

In [ ]:
plt.hist(c["dist_ln"],bins=20)
plt.show

In [ ]:
coolwarm = cm.get_cmap('coolwarm', 12)
print(coolwarm)

In [ ]:
# plot all HRUs in white (background color)
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
c.plot(column='dist_ln', cmap='coolwarm', ax=ax, legend=True)
#hrus.plot(column='tavgc_2013_10_1', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
# plot all HRUs in white (background color)
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
c.plot(column='dist', cmap='coolwarm', ax=ax, legend=True)
#hrus.plot(column='tavgc_2013_10_1', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
# Save the mapping shapefile
c.to_file(hrus_v10_to_v11_mapping_path)